In [156]:
from IPython.display import display, clear_output
import ipywidgets as widgets
import pandas as pd

In [157]:
#convert a string that looks like a list or dict into an actual Python object
#ast stands for Abstract Syntax Tree.
import ast 

In [158]:
titles_df = pd.read_csv('titles.csv')
print(titles_df.head())

         id                                title   type  \
0  ts300399  Five Came Back: The Reference Films   SHOW   
1   tm84618                          Taxi Driver  MOVIE   
2  tm154986                          Deliverance  MOVIE   
3  tm127384      Monty Python and the Holy Grail  MOVIE   
4  tm120801                      The Dirty Dozen  MOVIE   

                                         description  release_year  \
0  This collection includes 12 World War II-era p...          1945   
1  A mentally unstable Vietnam War veteran works ...          1976   
2  Intent on seeing the Cahulawassee River before...          1972   
3  King Arthur, accompanied by his squire, recrui...          1975   
4  12 American military prisoners in World War II...          1967   

  age_certification  runtime                                       genres  \
0             TV-MA       51                            ['documentation']   
1                 R      114                           ['drama', 'crim

In [159]:
titles_df['genres'] = titles_df['genres'].apply(ast.literal_eval)

In [160]:
#map genres to mood
genres_to_moods = {
    "comedy": "happy",
    "romance": "romantic",
    "drama": "sad",
    "thriller": "thrilling",
    "action": "exciting",
    "horror": "scary",
    "biography": "inspirational",
    "documentation": "inspirational",
    "animation": "happy",
    "fantasy": "exciting",
    "crime": "thrilling",
    "war": "sad",
    "adventure": "exciting"
}

In [161]:
#function to get genres by mood
def get_moods(genres):
    moods = {genres_to_moods[g.lower()] for g in genres if g.lower() in genres_to_moods}
    return list(moods) if moods else ['other']     

In [162]:
# Apply the function
titles_df['moods'] = titles_df['genres'].apply(get_moods)
# Show results
titles_df[['title', 'genres', 'moods','release_year','imdb_score','tmdb_score']].head(20)

,title,genres,moods,release_year,imdb_score,tmdb_score
0,Five Came Back: The Reference Films,[documentation],[inspirational],1945,NaN,NaN
1,Taxi Driver,"[drama, crime]","[sad, thrilling]",1976,8.2,8.179
2,Deliverance,"[drama, action, thriller, european]","[sad, exciting, thrilling]",1972,7.7,7.300
3,Monty Python and the Holy Grail,"[fantasy, action, comedy]","[exciting, happy]",1975,8.2,7.811
4,The Dirty Dozen,"[war, action]","[sad, exciting]",1967,7.7,7.600
5,Monty Python's Flying Circus,"[comedy, european]",[happy],1969,8.8,8.306
6,Life of Brian,[comedy],[happy],1979,8.0,7.800
7,Dirty Harry,"[thriller, action, crime]","[exciting, thrilling]",1971,7.7,7.500
8,Bonnie and Clyde,"[crime, drama, action]","[sad, exciting, thrilling]",1967,7.7,7.500
9,The Blue Lagoon,"[romance, action, drama]","[exciting, sad, romantic]",1980,5.8,6.156


In [166]:
#recommendation function
def get_movies_by_mood_with_rating (moods, top = 15):
    filtered = titles_df[titles_df['moods'].apply(lambda mood_list: moods in mood_list) & (titles_df['release_year'] >=2008)]
    results = filtered[['title', 'genres', 'release_year', 'imdb_score']]
    results = results[results['imdb_score'] >= 7.0]
    results = results.sort_values(by='imdb_score', ascending=False).reset_index(drop=True)
    return results.head(top)

In [167]:
mood_options = sorted(set(m for m_list in titles_df['moods'] for m in m_list if m != 'other'))
mood_dropdown = widgets.Dropdown(
    options=[""] + mood_options,  # Add blank option
    description='Select Mood:',
    value=''  # default blank
)
#button to trigger recommendation
recommend_button = widgets.Button(description = "Recommend Movies")
output = widgets.Output()

In [168]:
#button on click
def on_button_click(b):
    output.clear_output()
    select_mood = mood_dropdown.value
    if not select_mood:
        with output:
            print ("⚠️ Please select a mood from the dropdown.")
    else:
        with output:
            results = get_movies_by_mood_with_rating(select_mood)
            if results.empty:
                print("No high-rated movies found for this mood.")
            else:
                display (results)
recommend_button.on_click(on_button_click)
display(mood_dropdown)
display(recommend_button)
display(output)

Dropdown(description='Select Mood:', options=('', 'exciting', 'happy', 'inspirational', 'romantic', 'sad', 'sc…

Button(description='Recommend Movies', style=ButtonStyle())

Output()